* Data from https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/water_equiv/historical/

In [8]:
#run(`wget --force-html -i https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/water_equiv/historical/`)
#run(`unzip "*.zip"`)

Process(`unzip '*.zip'`, ProcessExited(0))

In [12]:
using CSV, DataFrames,Dates,Statistics

In [19]:
CSV.File("data/Metadaten_Geographie_02380.txt",delim=";")|>DataFrame

Row,Stations_id,Stationshoehe,Geogr.Breite,Geogr.Laenge,von_datum,bis_datum,Stationsname
,Int64,Float64,Float64,Float64,Int64,Int64,String31
1,2380,126.0,52.1623,8.0423,18910801,19401231,"Iburg-Glane, Bad"
2,2380,122.0,52.1592,8.0466,19470101,19550302,"Iburg-Glane, Bad"
3,2380,122.0,52.1592,8.0466,19550303,19581231,"Iburg-Glane, Bad"
4,2380,122.0,52.1569,8.041,19590101,19640221,"Iburg-Glane, Bad"
5,2380,122.0,52.1592,8.0466,19640222,19660331,"Iburg-Glane, Bad"
6,2380,115.0,52.1611,8.0566,19660401,19711231,"Iburg-Glane, Bad"
7,2380,115.0,52.1533,8.0434,19720101,20170531,"Iburg-Glane, Bad"
8,2380,105.0,52.145,8.0456,20170927,20200102,"Iburg-Glane, Bad"


In [32]:
christmas_max=DataFrame(lat=Float64[],long=Float64[],height=Float64[],val_max=Float64[],val_min=Float64[],val_mean=Float64[])
for file in readdir("data")
    if !startswith(file,"produkt_waequi")
        continue
    end
    station_num=file[end-8:end-4]
    df=CSV.File("data/$file",delim=';')|>DataFrame
    df.MESS_DATUM=map(x->(num=digits(x);Date(num[8]*1000+num[7]*100+num[6]*10+num[5],num[4]*10+num[3],num[2]*10+num[1])),df.MESS_DATUM)
    df=df[df.SH_TAG.!=-999,:]
    ch_vals=df[(day.(df.MESS_DATUM).==24).&(month.(df.MESS_DATUM).==12),:SH_TAG]
    if size(ch_vals,1)==0
        continue
    end
    df_info=CSV.File("data/Metadaten_Geographie_$(station_num).txt",delim=";")|>DataFrame
    breite=mean(df_info."Geogr.Breite")
    laenge=mean(df_info."Geogr.Laenge")
    hoehe=mean(df_info.Stationshoehe)
    christmas_max=vcat(christmas_max,DataFrame(lat=breite,long=laenge,height=hoehe,val_max=maximum(ch_vals),val_min=minimum(ch_vals),val_mean=mean(ch_vals)))

end

In [33]:
christmas_max

Row,lat,long,height,val_max,val_min,val_mean
,Float64,Float64,Float64,Float64,Float64,Float64
1,47.4004,10.2767,807.94,100.0,0.0,20.7885
2,49.953,11.5657,349.357,28.0,0.0,2.43529
3,47.6262,10.8068,798.0,74.0,0.0,12.0759
4,47.6,12.85,804.0,67.0,0.0,19.8286
5,47.7,11.8833,780.0,53.0,0.0,11.4118
6,48.6923,11.9022,470.0,30.0,0.0,4.03333
7,49.1068,9.72415,347.364,20.0,0.0,2.29268
8,49.0299,11.6029,367.5,0.0,0.0,0.0
9,48.7329,10.7745,452.0,5.0,0.0,2.6


In [5]:
using GLMakie

In [35]:
surface(christmas_max.long,christmas_max.lat,christmas_max.height,color=christmas_max.val_max)

In [8]:
fig=Figure()
ax=Axis(fig[1,1])
contourf!(ax,christmas_max.long,christmas_max.lat, christmas_max.val_max)
fig

In [46]:
using GeoMakie,ArchGDAL

In [47]:
layer=ArchGDAL.getlayer(ArchGDAL.read("../nuts5000_12-31.gk3.shape/nuts5000/NUTS5000_N1.shp"),0)
bounds=DataFrame(layer);
ArchGDAL.createcoordtrans(ArchGDAL.importURL("https://sg.geodatenzentrum.de/web_public/gdz/dokumentation/crs/gk3.prj"), ArchGDAL.importPROJ4("+proj=longlat +datum=WGS84 +no_defs +type=crs")) do transform
    for x in eachrow(bounds)
    ArchGDAL.transform!(x."", transform)
    end
end

In [321]:
fig=Figure()
g=GridLayout(fig[1,1:3])
Label(g[1,1],text="White christmas in Germany, where you have the best chance for it",fontsize=30)
Label(g[2,1],text="Based on triangulated station data",fontsize=25)
ax=GeoAxis(fig[2,1],latlims=(46.9,55.1),lonlims=(5.5,15.5),title="Maximum snow depth at christmas")
t=tricontourf!(ax,christmas_max.long, christmas_max.lat,christmas_max.val_max,colorrange=(0,50),colormap=:grays, levels=0:5:50)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(bounds."")),color=:transparent,strokecolor=:white,strokewidth=1)
scatter!(christmas_max.long, christmas_max.lat,marker='*',color=:green,markersize=20)
hidedecorations!(ax)
hidespines!(ax)
Colorbar(fig[3,1],t,vertical=false,label="Snow depth in cm")
ax=GeoAxis(fig[2,2],latlims=(46.9,55.1),lonlims=(5.5,15.5),title="Minimal snow depth at christmas")
t=tricontourf!(ax,christmas_max.long, christmas_max.lat,christmas_max.val_min,colorrange=(0,50),colormap=:grays, levels=0:1:5)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(bounds."")),color=:transparent,strokecolor=:white,strokewidth=1)
scatter!(christmas_max.long, christmas_max.lat,marker='*',color=:green,markersize=20)
hidedecorations!(ax)
hidespines!(ax)
Colorbar(fig[3,2],t,vertical=false,label="Snow depth in cm")

ax=GeoAxis(fig[2,3],latlims=(46.9,55.1),lonlims=(5.5,15.5),title="Mean snow depth at christmas")
t=tricontourf!(ax,christmas_max.long, christmas_max.lat,christmas_max.val_mean,colorrange=(0,10),colormap=:grays, levels=0:1:10)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(bounds."")),color=:transparent,strokecolor=:white,strokewidth=1)
scatter!(christmas_max.long, christmas_max.lat,marker='*',color=:green,markersize=20)
hidedecorations!(ax)
hidespines!(ax)
Colorbar(fig[3,3],t,vertical=false,label="Snow depth in cm")
Label(fig[4,1:3],text="Data: © Deutscher Wetterdienst\nGeodata: © GeoBasis-DE / BKG (2023)",justification=:left)
e1=MarkerElement(color = :green, marker = '*', markersize = 20)
Legend(fig[2,4],[e1],["Station"],patchsize = (35, 35), rowgap = 10,framevisible=false)
Makie.resize_to_layout!(fig)
fig

In [318]:
save("white_christmas_black_white.png",fig,px_per_unit=2)

In [1]:
using DelimitedFiles

In [2]:
dgm=readdlm("dgm1000_utm32s.asc")[7:end,:]

866×643 Matrix{Any}:
 -9999.0  -9999  -9999  -9999  -9999  …  -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999  …  -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
     ⋮                                ⋱                    ⋮         
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999    -9999  -9999  -9999  -9999     -9999  -9999  -9999  -9999  -9999
 -9999   

In [8]:
dgm[dgm.<=-999].=NaN

1-element view(::Vector{Any}, [177626]) with eltype Any:
 NaN

In [62]:
using DelaunayTriangulation

In [165]:
DEU_arch = ArchGDAL.getlayer(ArchGDAL.read("../ne_110m_admin_0_countries_DEU.geojson"),0)|>DataFrame

Row,,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,ADM0_A3,GEOU_DIF,GEOUNIT,GU_A3,SU_DIF,SUBUNIT,SU_A3,BRK_DIFF,NAME,NAME_LONG,BRK_A3,BRK_NAME,BRK_GROUP,ABBREV,POSTAL,FORMAL_EN,FORMAL_FR,NAME_CIAWF,NOTE_ADM0,NOTE_BRK,NAME_SORT,NAME_ALT,MAPCOLOR7,MAPCOLOR8,MAPCOLOR9,MAPCOLOR13,POP_EST,POP_RANK,POP_YEAR,GDP_MD,GDP_YEAR,ECONOMY,INCOME_GRP,FIPS_10,ISO_A2,ISO_A2_EH,ISO_A3,ISO_A3_EH,ISO_N3,ISO_N3_EH,UN_A3,WB_A2,WB_A3,WOE_ID,WOE_ID_EH,WOE_NOTE,ADM0_ISO,ADM0_DIFF,ADM0_TLC,ADM0_A3_US,ADM0_A3_FR,ADM0_A3_RU,ADM0_A3_ES,ADM0_A3_CN,ADM0_A3_TW,ADM0_A3_IN,ADM0_A3_NP,ADM0_A3_PK,ADM0_A3_DE,ADM0_A3_GB,ADM0_A3_BR,ADM0_A3_IL,ADM0_A3_PS,ADM0_A3_SA,ADM0_A3_EG,ADM0_A3_MA,ADM0_A3_PT,ADM0_A3_AR,ADM0_A3_JP,ADM0_A3_KO,ADM0_A3_VN,ADM0_A3_TR,ADM0_A3_ID,ADM0_A3_PL,ADM0_A3_GR,ADM0_A3_IT,ADM0_A3_NL,ADM0_A3_SE,ADM0_A3_BD,ADM0_A3_UA,ADM0_A3_UN,ADM0_A3_WB,CONTINENT,REGION_UN,SUBREGION,REGION_WB,NAME_LEN,LONG_LEN,ABBREV_LEN,⋯
,IGeometr…,String,Int32,Int32,String,String,Int32,Int32,String,String,String,String,Int32,String,String,Int32,String,String,Int32,String,String,String,String,Missing,String,String,String,Missing,String,Missing,Missing,String,Missing,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,String,String,String,String,String,String,String,String,String,String,String,String,Int32,Int32,String,String,Missing,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,Int32,Int32,String,String,String,String,Int32,Int32,Int32,⋯
1,Geometry: wkbPolygon,Admin-0 country,1,2,Germany,DEU,0,2,Sovereign country,1,Germany,DEU,0,Germany,DEU,0,Germany,DEU,0,Germany,Germany,DEU,Germany,missing,Ger.,D,Federal Republic of Germany,missing,Germany,missing,missing,Germany,missing,2,5,5,1,83132799,16,2019,3861123,2019,1. Developed region: G7,1. High income: OECD,GM,DE,DE,DEU,DEU,276,276,276,DE,DEU,23424829,23424829,Exact WOE match as country,DEU,missing,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,DEU,-99,-99,Europe,Europe,Western Europe,Europe & Central Asia,7,7,4,⋯


In [204]:
coords=GeoInterface.coordinates.(GeoMakie.to_multipoly.(GeoMakie.geo2basic.(DEU_arch."")))

1-element Vector{Vector{Vector{Vector{Vector{Float64}}}}}:
 [[[[14.119686, 53.757029], [14.353315, 53.248171], [14.074521, 52.981263], [14.4376, 52.62485], [14.685026, 52.089947], [14.607098, 51.745188], [15.016996, 51.106674], [14.570718, 51.002339], [14.307013, 51.117268], [14.056228, 50.926918]  …  [8.526229, 54.962744], [9.282049, 54.830865], [9.921906, 54.983104], [9.93958, 54.596642], [10.950112, 54.363607], [10.939467, 54.008693], [11.956252, 54.196486], [12.51844, 54.470371], [13.647467, 54.075511], [14.119686, 53.757029]]]]

In [206]:
coord_tri=map(x->tuple(x[1],x[2]),vcat(vcat(vcat(coords...)...)...))


58-element Vector{Tuple{Float64, Float64}}:
 (14.119686, 53.757029)
 (14.353315, 53.248171)
 (14.074521, 52.981263)
 (14.4376, 52.62485)
 (14.685026, 52.089947)
 (14.607098, 51.745188)
 (15.016996, 51.106674)
 (14.570718, 51.002339)
 (14.307013, 51.117268)
 (14.056228, 50.926918)
 ⋮
 (9.282049, 54.830865)
 (9.921906, 54.983104)
 (9.93958, 54.596642)
 (10.950112, 54.363607)
 (10.939467, 54.008693)
 (11.956252, 54.196486)
 (12.51844, 54.470371)
 (13.647467, 54.075511)
 (14.119686, 53.757029)

In [262]:
first_pts=tuple.(christmas_max.long,christmas_max.lat)

res_pts=Point2f[]
for x in first_pts
if ArchGDAL.within(ArchGDAL.createpoint(x),DEU_arch.""[1])
    push!(res_pts,Point2f(x))
end
end

In [264]:
boundary_nodes, pts = convert_boundary_points_to_indices(coord_tri; existing_points=res_pts)

([896, 897, 898, 899, 900, 901, 902, 903, 904, 905  …  944, 945, 946, 947, 948, 949, 950, 951, 952, 896], Point{2, Float32}[[10.27675, 47.400375], [11.565728, 49.952957], [10.8067665, 47.626232], [12.85, 47.6], [11.8833, 47.7], [11.9022, 48.6923], [9.724146, 49.10682], [11.6029, 49.02995], [10.7745, 48.7329], [10.2469, 49.2553]  …  [8.572118, 54.395645], [8.526229, 54.962746], [9.282049, 54.830864], [9.921906, 54.983105], [9.93958, 54.59664], [10.950112, 54.363605], [10.939467, 54.008694], [11.956252, 54.196487], [12.51844, 54.47037], [13.647467, 54.075512]])

In [265]:
pts=reverse(pts)

952-element Vector{Point{2, Float32}}:
 [13.647467, 54.075512]
 [12.51844, 54.47037]
 [11.956252, 54.196487]
 [10.939467, 54.008694]
 [10.950112, 54.363605]
 [9.93958, 54.59664]
 [9.921906, 54.983105]
 [9.282049, 54.830864]
 [8.526229, 54.962746]
 [8.572118, 54.395645]
 ⋮
 [10.7745, 48.7329]
 [11.6029, 49.02995]
 [9.724146, 49.10682]
 [11.9022, 48.6923]
 [11.8833, 47.7]
 [12.85, 47.6]
 [10.8067665, 47.626232]
 [11.565728, 49.952957]
 [10.27675, 47.400375]

In [266]:
boundary_nodes=reverse(boundary_nodes)

58-element Vector{Int64}:
 896
 952
 951
 950
 949
 948
 947
 946
 945
 944
   ⋮
 904
 903
 902
 901
 900
 899
 898
 897
 896

In [269]:
cons_tri = triangulate(pts; boundary_nodes, delete_ghosts = false)

Delaunay Triangulation.
    Constrained: true
    Has ghost triangles: false
    Number of points: 952
    Number of triangles: 1610
    Number of edges: 2655

In [270]:
fig, ax, sc = triplot(cons_tri)
scatter!(ax,pts)
fig

In [273]:
cons_tri = triangulate(pts; delete_ghosts = false)


Delaunay Triangulation.
    Constrained: false
    Has ghost triangles: true
    Number of points: 952
    Number of triangles: 1902
    Number of edges: 2853